# Lesson 8: Advanced Queries with Subqueries

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-7  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Write subqueries in WHERE clauses
- Use subqueries in SELECT statements
- Work with IN, NOT IN, EXISTS, and NOT EXISTS
- Combine subqueries with JOINs
- Solve complex data retrieval problems
- Understand when to use subqueries vs JOINs


## 📚 What are Subqueries?

A **subquery** (or nested query) is a query inside another query. They allow you to:
- Use the results of one query in another
- Break complex problems into simpler steps
- Write more readable code
- Solve problems that would be difficult with just JOINs

### Real-World Example:

**Question:** "Find all characters from the same planet as Luke Skywalker"

**Without Subquery (2 steps):**
1. Query: What planet is Luke from? → Answer: Tatooine
2. Query: Who else is from Tatooine?

**With Subquery (1 step):**
```sql
SELECT name FROM characters 
WHERE homeworld_id = (SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker');
```

### Think of it Like:

Subqueries are like asking a question inside a question:
- "Show me students taller than **[the average height]**"
- The bracketed part is calculated first (subquery)
- Then used in the main query


## 🛠️ Setup: Connect to Database

**Run the next 3 cells:**


In [ ]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [ ]:
%sql sqlite:///starwars.db

## 🔍 Part 1: Subqueries in WHERE Clause

### Basic Subquery Syntax

```sql
SELECT columns
FROM table_name
WHERE column OPERATOR (SELECT column FROM table WHERE condition);
```

### Query 1: Find Luke's Homeworld First

**Step 1:** See what the subquery returns:


In [ ]:
%%sql
-- Simple subquery: Characters taller than average


### Query 2: Use That in a Subquery

Now find all characters from the same planet:


In [ ]:
%%sql
-- Subquery in FROM clause


**Explanation:**

1. Subquery executes first: `(SELECT homeworld_id FROM characters WHERE name = 'Luke Skywalker')`
2. Returns a single value (e.g., 1)
3. Main query becomes: `WHERE homeworld_id = 1`
4. Finds all matches

**Key:** Subquery with `=` must return exactly ONE value.


### Query 3: Subquery with Comparison Operators

Find characters taller than average:


In [ ]:
%%sql
-- Subquery with IN operator


**Breakdown:**

1. Subquery: `(SELECT AVG(height) FROM characters)` → calculates average
2. Main query: Finds everyone taller than that average

**Alternative without subquery:** Would require calculating average separately, storing it, then querying.


### Query 4: Subquery with IN Operator

**IN** allows subqueries that return multiple values:


In [ ]:
%%sql
-- Subquery with EXISTS


**Explanation:**

1. Subquery finds all planet IDs with desert terrain (returns multiple IDs)
2. `IN` checks if homeworld_id matches ANY of those IDs
3. Returns characters from any desert planet

**Why IN?** Because the subquery returns multiple rows (multiple desert planets).


### Query 5: Subquery with NOT IN

**NOT IN** excludes values:


In [ ]:
%%sql
-- Correlated subquery


### Query 6: Nested Subqueries

Subqueries can be nested inside other subqueries:


In [ ]:
%%sql
-- Nested subquery


**This combines:**
- JOIN (to get planet names)
- Subquery (to calculate average population)
- WHERE (to filter)


## ✅ Part 2: EXISTS and NOT EXISTS

**EXISTS** checks if a subquery returns any rows. It's often more efficient than IN for large datasets.

### EXISTS vs IN

| Feature | EXISTS | IN |
|---------|--------|-----|
| Returns | TRUE/FALSE | Values to match |
| Performance | Usually faster | Can be slower |
| Readability | Clear intent | Also clear |
| Best for | Existence checks | Value matching |

### EXISTS Syntax

```sql
SELECT columns
FROM table1
WHERE EXISTS (SELECT 1 FROM table2 WHERE condition);
```

**Note:** `SELECT 1` is convention (just checks for existence, doesn't return actual data)

### Query 7: Using EXISTS


In [ ]:
%%sql
-- Simple CASE: Categorize heights


**Explanation:**

For each character:
1. Check if there's ANY row in character_vehicles with their ID
2. If yes (EXISTS returns TRUE), include them
3. If no, skip them

**Correlated Subquery:** The subquery references the outer query's table (`c.id`).


### Query 8: Using NOT EXISTS

Find characters who DON'T pilot any vehicles:


In [ ]:
%%sql
-- CASE with aggregation


**Alternative with LEFT JOIN:**

```sql
-- Same result using LEFT JOIN
SELECT c.name, c.species
FROM characters c
LEFT JOIN character_vehicles cv ON c.id = cv.character_id
WHERE cv.character_id IS NULL;
```

**NOT EXISTS is often:**
- More readable (clearer intent)
- Better performing
- Standard SQL approach


### Query 9: EXISTS with Correlated Subquery

Find planets that have at least one human:


In [ ]:
%%sql
-- CASE in ORDER BY


**Correlated subquery explained:**

For each planet in the outer query:
1. Check if there's a character with `homeworld_id = p.id` AND `species = 'Human'`
2. The subquery "sees" the outer query's `p.id`
3. This correlation happens row-by-row


### Query 10: Complex EXISTS Example

Find characters who pilot the same type of vehicle as Luke:


In [ ]:
%%sql
-- UNION: Combine query results


**This is complex!** Let's break it down:

1. **Outer query:** Loop through all characters
2. **EXISTS subquery:** For each character, check if they pilot...
3. **...a vehicle_class that's IN:**
4. **Nested subquery:** The vehicle classes Luke pilots
5. **AND c.name != 'Luke':** Don't include Luke himself

**Real-world translation:** "Show me everyone except Luke who flies the same type of vehicle as Luke.


## 📊 Part 3: Subqueries in SELECT

You can use subqueries in the SELECT list to add calculated columns.

### Query 11: Subquery as a Column


In [ ]:
%%sql
-- UNION ALL: Keep duplicates


**Explanation:**

For each character:
1. Main query gets name and species
2. Subquery counts their vehicles
3. Result appears as a column

**Alternative:** Could use LEFT JOIN + GROUP BY, but subquery is sometimes clearer.


### Query 12: Multiple Subqueries in SELECT

Show how each character compares to average:


In [ ]:
%%sql
-- UNION with sorting


**Shows:** Who's tallest/shortest compared to average.

**Note:** The subquery runs for each row. For large datasets, consider calculating once and storing in a variable.


## 📊 Subquery vs JOIN: When to Use What?

### Use Subqueries When:

- ✅ You need a single aggregate value (AVG, MAX, MIN)
- ✅ Checking for existence (EXISTS/NOT EXISTS)
- ✅ Query logic is clearer with nesting
- ✅ You need calculated columns

### Use JOINs When:

- ✅ Combining columns from multiple tables
- ✅ Performance is critical
- ✅ You need multiple columns from related tables
- ✅ Working with very large datasets

### Example Comparison:


In [ ]:
%%sql
-- Common Table Expression (CTE) example


In [ ]:
%%sql
-- Multiple CTEs


**Both produce same result!** Choose based on:
- Readability
- Performance needs
- Personal/team preference


## 🎓 Practice Exercises

### Exercise 1: Subquery with IN


In [ ]:
%%sql
-- Exercise 1: Subquery to find shortest character per species


### Exercise 2: NOT EXISTS


In [ ]:
%%sql
-- Exercise 2: CASE to label planet climates


### Exercise 3: Subquery with Comparison


In [ ]:
%%sql
-- Exercise 3: UNION to combine characters and planets


### Exercise 4: Complex Nested Subquery


In [ ]:
%%sql
-- Exercise 4: CTE to find popular planets


## 🐛 Common Errors & Troubleshooting

### Error: "Subquery returns more than 1 row"

**Problem:** Using `=` with a subquery that returns multiple values.

**Wrong:**
```sql
WHERE homeworld_id = (SELECT id FROM planets WHERE climate = 'temperate');
-- Error! Multiple temperate planets
```

**Correct:**
```sql
WHERE homeworld_id IN (SELECT id FROM planets WHERE climate = 'temperate');
-- IN handles multiple values
```

**Rule:** 
- Use `=` when subquery returns ONE value
- Use `IN` when subquery returns MULTIPLE values


### Slow Query Performance

**Problem:** Correlated subquery runs for every row.

**Less Efficient:**
```sql
SELECT name,
    (SELECT COUNT(*) FROM vehicles WHERE pilot_id = c.id)
FROM characters c;
-- Subquery runs once per character
```

**More Efficient:**
```sql
SELECT c.name, COUNT(v.id) AS vehicle_count
FROM characters c
LEFT JOIN vehicles v ON v.pilot_id = c.id
GROUP BY c.name;
-- JOIN runs once
```

**When to use each:**
- Subqueries: Clearer logic, small datasets
- JOINs: Better performance, large datasets


### Correlated Subquery Confusion

**Problem:** Forgetting to reference outer query.

**Wrong:**
```sql
WHERE EXISTS (SELECT 1 FROM planets WHERE population > 1000000);
-- Doesn't reference outer query! Always TRUE or FALSE for everyone
```

**Correct:**
```sql
WHERE EXISTS (
    SELECT 1 
    FROM planets p 
    WHERE c.homeworld_id = p.id 
    AND p.population > 1000000
);
-- Properly correlated with c.homeworld_id
```


### NULL Handling in Subqueries

**Problem:** NULLs cause unexpected results.

**Example:**
```sql
WHERE height > (SELECT AVG(height) FROM characters);
-- If any height is NULL, AVG might be unexpected
```

**Solution:**
```sql
WHERE height > (
    SELECT AVG(height) 
    FROM characters 
    WHERE height IS NOT NULL
);
-- Explicitly filter NULLs
```

**Rule:** Always consider NULLs in subqueries!


## 🎯 Challenge Problem

**Task:** Write a query using ONLY subqueries (no JOINs) that finds all characters who:

1. Are from the same planet as someone who pilots a starfighter
2. But do NOT pilot any starfighters themselves

**Requirements:**
- Use subqueries only
- Use IN or EXISTS
- Filter correctly

Try it yourself first!


### Challenge Solution


In [ ]:
%%sql
-- Challenge: Complex query combining techniques


**Breakdown:**

1. **Outermost WHERE:** Character must be from the right planet AND not pilot starfighters
2. **First IN subquery:** Find homeworld IDs of starfighter pilots
3. **Second NOT IN subquery:** Exclude anyone who pilots starfighters
4. **Nested subqueries:** Find starfighter IDs at multiple levels

**This demonstrates:** Complex logic possible with just subqueries!


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Write subqueries in WHERE clauses (with =, >, <, IN, NOT IN)
- ✅ Use EXISTS and NOT EXISTS for existence checks
- ✅ Write subqueries in SELECT statements for calculated columns
- ✅ Understand correlated vs non-correlated subqueries
- ✅ Nest subqueries multiple levels deep
- ✅ Know when to use subqueries vs JOINs
- ✅ Handle NULL values in subqueries
- ✅ Optimize subquery performance

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| Subquery in WHERE | Filter using nested query | `WHERE id = (SELECT...)` |
| `IN` | Match any value in list | `WHERE id IN (SELECT...)` |
| `NOT IN` | Exclude values in list | `WHERE id NOT IN (SELECT...)` |
| `EXISTS` | Check if subquery returns rows | `WHERE EXISTS (SELECT...)` |
| `NOT EXISTS` | Check if subquery is empty | `WHERE NOT EXISTS (SELECT...)` |
| Subquery in SELECT | Calculate column value | `SELECT (SELECT COUNT(*) FROM...)` |
| Correlated Subquery | References outer query | `WHERE EXISTS (SELECT 1 WHERE outer.id = inner.id)` |

### Subquery vs JOIN Decision Tree

**Question 1:** Do I need columns from multiple tables?
- **Yes** → Consider JOIN
- **No** → Subquery might be clearer

**Question 2:** Am I checking for existence?
- **Yes** → Use EXISTS/NOT EXISTS
- **No** → Continue

**Question 3:** Do I need a single calculated value?
- **Yes** → Subquery in WHERE (e.g., AVG, MAX)
- **No** → Continue

**Question 4:** Is performance critical?
- **Yes** → Prefer JOIN
- **No** → Choose based on readability

## 🎉 Outstanding Work!

You can now write complex nested queries and solve advanced data retrieval problems! In the next lesson, you'll integrate SQL with Python to build database-driven applications.

**Ready to continue?** Open `lesson9_database.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson8_advanced.ipynb
git commit -m "Completed Lesson 8: Advanced queries with subqueries"
git push
```
